In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/MyDrive/

/content/drive/MyDrive


In [4]:
ls

'Colab Notebooks'/  'NLP '/


In [5]:
cd NLP\ 

/content/drive/MyDrive/NLP 


In [6]:
ls

Classification/


In [7]:
cd Classification/

/content/drive/MyDrive/NLP /Classification


In [8]:
ls

dataset/  dataset.zip


In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
replace dataset/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
ls

dataset/  dataset.zip


In [10]:
cd dataset/

/content/drive/MyDrive/NLP /Classification/dataset


In [11]:
ls

sample_submission.csv  test.csv  train.csv


In [12]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [13]:
dataset = pd.read_csv('train.csv')

In [14]:
dataset.head()

,video_id,category_id,description
0,7NcVNt7zQJM,10.0,Music video by Luke Bryan performing Out Of No...
1,hNsd-m30m2E,24.0,thanks for watching!!! happy holidays everyone...
2,yLdav-8af6g,1.0,Marcus and Adam build their own custom lightsa...
3,Ii3Rr4-Pxow,26.0,Make sure you subscribe to my channel and hit ...
4,515lLZS082U,24.0,It's a riff-off pitches! Get Tix: http://unvrs...


In [15]:
dataset.drop(['video_id'], axis = 1, inplace = True)

In [16]:
dataset.dropna(inplace = True)

In [17]:
dataset.tail()

,category_id,description
3466,24.0,Subscribe now for more! http://bit.ly/1KyA9sV\...
3467,28.0,Get a free audio book by clicking http://www.a...
3468,10.0,The Official video for Moby's Like A Motherles...
3469,26.0,"Hey Larlees, todays video is a fun collab with..."
3470,22.0,http://billwurtz.com\n\n\n\npatreon: http://p...


Preprocessing

In [45]:
nltk.download('stopwords') 
corpus = [] 
ps = PorterStemmer() 
for i in range(len(dataset)): 
   #Removing special characters
   text = re.sub('[^a-zA-Z]', ' ', dataset['description'][i]).lower().split()
   #.lower()
   #Stemming and removing stop words
   text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
   #lower
   text = [entry.lower() for entry in text]
   #Joining all the cleaned words to form a sentence
   text = ' '.join(text)
   #Adding the cleaned sentence to a list
   corpus.append(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Generate count Vector


In [46]:
cv = CountVectorizer(max_features = 120)
X = cv.fit_transform(corpus).toarray()
y = dataset['category_id'].values

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [48]:
classifier = SVC()
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [49]:
y_pred = classifier.predict(X_val)

In [50]:
def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

#Creating the confusion matrix with y_val and y_pred
cm = confusion_matrix(y_val, y_pred)

print("Accuracy : ", accuracy(cm))

Accuracy :  0.472168905950096


In [51]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [52]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_val)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_val)*100)

Naive Bayes Accuracy Score ->  42.32245681381958


In [53]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train, y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_val)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_val)*100)

SVM Accuracy Score ->  62.18809980806142


In [61]:
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier

bagClassifier = BaggingClassifier(n_jobs=-1)
bagClassifier.fit(X_train, y_train)
bagPreds = bagClassifier.predict(X_val)
# Use accuracy_score function to get the accuracy
print("Bagging classifier Accuracy Score -> ",accuracy_score(bagPreds, y_val)*100)

Bagging classifier Accuracy Score ->  73.70441458733205


In [62]:
from sklearn.naive_bayes import MultinomialNB

nbClassifier = MultinomialNB()
nbClassifier.fit(X_train, y_train)
nbPreds = nbClassifier.predict(X_val)
print("gradient naive_bayes classifier Accuracy Score -> ",accuracy_score(nbPreds, y_val)*100)

gradient naive_bayes classifier Accuracy Score ->  42.32245681381958


In [59]:
from sklearn.ensemble import GradientBoostingClassifier

boostClassifier = GradientBoostingClassifier()
boostClassifier.fit(X_train, y_train)
boostPreds = boostClassifier.predict(X_val)
# Use accuracy_score function to get the accuracy
print("gradient booster classifier Accuracy Score -> ",accuracy_score(boostPreds, y_val)*100)

gradient booster classifier Accuracy Score ->  66.41074856046065


In [64]:
from sklearn.ensemble import RandomForestClassifier
rfClassifier = RandomForestClassifier(n_jobs=-1)
rfClassifier.fit(X_train, y_train)
rfPreds = rfClassifier.predict(X_val)
print("random forest classifier Accuracy Score -> ",accuracy_score(rfPreds, y_val)*100)

random forest classifier Accuracy Score ->  77.2552783109405


In [65]:
from sklearn.tree import DecisionTreeClassifier

dtClassifier = DecisionTreeClassifier()
dtClassifier.fit(X_train, y_train)
dtPreds = dtClassifier.predict(X_val)
print("decision tree classifier Accuracy Score -> ",accuracy_score(rfPreds, y_val)*100)

decision tree classifier Accuracy Score ->  77.2552783109405


In [66]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

knnClf = KNeighborsClassifier()

knnClf.fit(X_train, y_train)
knnPredictions = knnClf.predict(X_val)
print("kneighbors classifier Accuracy Score -> ",accuracy_score(rfPreds, y_val)*100)

kneighbors classifier Accuracy Score ->  77.2552783109405
